<a href="https://colab.research.google.com/github/Renan-Domingues/LearnTheBasics-Pytorch/blob/main/Tutorials_00_Quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# pytorch offers libraries such as TorchText, TorchVision and TorchAudio, all of whitch includes datasets.
#For this tutorial we will be using a TorchVision dataset.


The Torchvision.datasets modules contain Dataset objects for many real-word vision data.Full list: https://pytorch.org/vision/stable/datasets.html
In this tutorial , we use the FashionMNIST dataset: https://pytorch.org/vision/stable/generated/torchvision.datasets.FashionMNIST.html#torchvision.datasets.FashionMNIST

In [ ]:
# Download training data from open datasets

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open dataset

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


100%|██████████| 26421880/26421880 [00:03<00:00, 8311449.76it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 139029.45it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 2606729.66it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 6589037.84it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



We pass the Dataset as an argument to DataLoader. This wraps an iterable over our dataset, and suports automatic batching, sampling, shuffling and multiprocess data loading.

In the next step I'm goin to define a bach size of 64.
it meand that each element in the dataloader interable will return batch of 64 features and labels


In [ ]:
batch_size = 64

# Create a data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


### Creating Models
We create a class that inheris from nn.Module.
we will define the layers of the networn in the __init__ function
and specify how data will pass through the network in the foward function

In [ ]:
# Get cpu, gpu or mps device for training

device = ("cuda"
if torch.cuda.is_available()
else "mps"
if torch.backends.mps.is_available()
else "cpu")
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512, 10)
  )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### Optimizing the Model Parameters


In [ ]:
# To train a model, we need a loss function and an optimizer.

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
'''
in a single loop, the model makes predictions on the training dataset (fed to it in batches),
and backpropagates the prediction error to adjust the model's parameters
'''

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [ ]:
'''We also check the model's performance against the test dataset
to ensure it is learning '''

def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0,0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (epochs). During each epoch, the model learns parameters to make better predictions.
We print the model's accuracy and loss at each epoch; we'd like to see the accutacy increase and the loss decrease with every epoch.


In [ ]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
  print("Done!")

Epoch 1
-------------------
loss: 2.308742 [   64/60000]
loss: 2.289044 [ 6464/60000]
loss: 2.276515 [12864/60000]
loss: 2.267895 [19264/60000]
loss: 2.253698 [25664/60000]
loss: 2.225273 [32064/60000]
loss: 2.232852 [38464/60000]
loss: 2.199456 [44864/60000]
loss: 2.200396 [51264/60000]
loss: 2.163410 [57664/60000]
Test Error: 
 Accuracy: 39.7%, Avg loss: 2.159931 

Done!
Epoch 2
-------------------
loss: 2.177898 [   64/60000]
loss: 2.161890 [ 6464/60000]
loss: 2.111350 [12864/60000]
loss: 2.125921 [19264/60000]
loss: 2.076976 [25664/60000]
loss: 2.017503 [32064/60000]
loss: 2.044367 [38464/60000]
loss: 1.967549 [44864/60000]
loss: 1.979645 [51264/60000]
loss: 1.901089 [57664/60000]
Test Error: 
 Accuracy: 57.2%, Avg loss: 1.905220 

Done!
Epoch 3
-------------------
loss: 1.943844 [   64/60000]
loss: 1.907855 [ 6464/60000]
loss: 1.803952 [12864/60000]
loss: 1.843039 [19264/60000]
loss: 1.726494 [25664/60000]
loss: 1.686010 [32064/60000]
loss: 1.702222 [38464/60000]
loss: 1.605964 [4

### Saving and Loading a model

In [ ]:
torch.save(model.state_dict(), "model.pht")
print("Saved Pytorch Model State to model.pht")

Saved Pytorch Model State to model.pht


In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pht"))

<All keys matched successfully>

### This model can now be used to make predictions.

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  x = x.to(device)
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'predicted: "{predicted}", Actual: "{actual}"')


predicted: "Ankle boot", Actual: "Ankle boot"
